In [2]:
!pip install scipy

   ---------------------------------------- 0.0/44.8 MB ? eta -:--:--
   --- ------------------------------------ 3.4/44.8 MB 20.0 MB/s eta 0:00:03
   ---------- ----------------------------- 11.8/44.8 MB 32.0 MB/s eta 0:00:02
   ------------------ --------------------- 21.0/44.8 MB 36.8 MB/s eta 0:00:01
   -------------------------- ------------- 29.4/44.8 MB 37.2 MB/s eta 0:00:01
   --------------------------------- ------ 37.7/44.8 MB 37.5 MB/s eta 0:00:01
   ---------------------------------------  44.6/44.8 MB 37.8 MB/s eta 0:00:01
   ---------------------------------------- 44.8/44.8 MB 35.6 MB/s eta 0:00:00


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
scikit-learn 1.4.2 requires joblib>=1.2.0, which is not installed.
scikit-learn 1.4.2 requires threadpoolctl>=2.0.0, which is not installed.
sktime 0.26.0 requires pandas<2.2.0,>=1.1, which is not installed.
sktime 0.26.0 requires scikit-base<0.8.0, which is not installed.
sktime 0.26.0 requires numpy<1.27,>=1.21, but you have numpy 2.1.1 which is incompatible.


In [1]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import mean_squared_error, r2_score

from xgboost import XGBRegressor
from xgboost import plot_importance
from xgboost import plot_tree
from xgboost import XGBClassifier

from patsy import dmatrices
import numpy as np
import pandas as pd

ModuleNotFoundError: No module named 'scipy'

In [15]:
dataset = pd.read_excel(r"C:\Users\JacopoBinati\OneDrive - Venionaire Capital\Desktop\crunchbase\analysis\analysis_data.xlsx")

In [16]:
pd.set_option('display.max_columns', None)

#### now all we need to do before running the models is to get only float, bool, int variables

In [17]:
columns_to_drop = [
    "Organization Name",
    "IPO Status",
    "Company Type",
    "Number of Employees",
    "Last Funding Type",
    "Last Funding Date",
    "Founded Date",
    "Founded Date Precision",
    "Operating Status",
    "Top 5 Investors",
    "Headquarters Location",
    "Industry Groups",
    "Industries",
    "Headquarters Regions",
    "Main Industry",
    "Industry Name"

]
dataset = dataset.drop(columns=columns_to_drop, axis=1)

#### Need to be dropped all the variables which are not dummies, integers or float

In [18]:
dataset

Number of Founders  Number of Funding Rounds  \
0                      1                         3   
1                      3                         4   
2                      1                         1   
3                      2                         1   
4                      1                         4   
...                  ...                       ...   
4670                   1                         1   
4671                   1                         3   
4672                   2                         2   
4673                   1                         1   
4674                   4                         6   

      Last Funding Amount (in USD)  Last Equity Funding Amount (in USD)  \
0                                0                                    0   
1                           280498                                    0   
2                                0                                    0   
3                                0                                    0   
4                                0                                    0   
...                            ...                                  ...   
4670                             0                                    0   
4671                      14232849                             14232849   
4672                       3600000                              3600000   
4673                       2887920                              2887920   
4674                             0                                    0   

      Total Equity Funding Amount (in USD)  Total Funding Amount (in USD)  \
0                                        0                        2192645   
1                                        0                         482615   
2                                        0                              0   
3                                        0                              0   
4                                        0                              0   
...                                    ...                            ...   
4670                                     0                              0   
4671                              58385810                       58385810   
4672                               3600000                        3600000   
4673                               2887920                        2887920   
4674                               8100000                        8110275   

      Number of Investors  Status__Private  CompanyType__For Profit  \
0                       1                1                        1   
1                       3                1                        1   
2                       3                1                        1   
3                       2                1                        1   
4                       3                1                        1   
...                   ...              ...                      ...   
4670                    1                1                        1   
4671                    8                1                        1   
4672                    4                1                        1   
4673                    1                1                        1   
4674                   13                1                        1   

      CompanyType__Non-profit  NumberEmployees__1-10  NumberEmployees__10001+  \
0                           0                      1                        0   
1                           0                      1                        0   
2                           0                      0                        0   
3                           0                      1                        0   
4                           0                      0                        0   
...                       ...                    ...                      ...   
4670                        0                      0                        1   
4671                        0 

## Get preliminary information with PyCaret for feature importance, SHAP and xgboost hypertuning

In [29]:
import pycaret

RuntimeError: ('Pycaret only supports python 3.9, 3.10, 3.11. Your actual Python version: ', sys.version_info(major=3, minor=8, micro=19, releaselevel='final', serial=0), 'Please UPGRADE your Python version.')

In [19]:
from pycaret.classification import *
clf1 = setup(data=dataset, target='Return on Equity', session_id=123)
best_model = compare_models()

ModuleNotFoundError: No module named 'pycaret'

In [6]:
X = dataset.drop('Return on Equity', axis=1)
y = dataset['Return on Equity']

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.26, random_state=42)


## XG Boost Classifier

In [8]:
param_grid = {
    'n_estimators': [400, 800],
    'max_depth': [6, 10],
    'learning_rate': [0.01, 0.1],
    'colsample_bytree': [0.1, 0.5]
}

# Initialize the XGBClassifier
xgb = XGBClassifier(objective='binary:logistic', random_state=42)

# Set up GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb,
    param_grid=param_grid,
    cv=5,
    scoring="accuracy",
    n_jobs=-1,
    verbose=2
)

# Fit the model
grid_search.fit(X_train, y_train)

# Print the best parameters
best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Get the best estimator
best_xgb = grid_search.best_estimator_

# Make predictions on the test set
y_pred = best_xgb.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy on test set: {accuracy:.4f}")

# Print classification report
print("\nClassification Report:")
print(classification_report(y_test, y_pred))


Fitting 5 folds for each of 16 candidates, totalling 80 fits


ValueError: 
All the 80 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
48 fits failed with the following error:
Traceback (most recent call last):
  File "c:\Users\JacopoBinati\anaconda3\envs\jacopo\lib\site-packages\sklearn\model_selection\_validation.py", line 686, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "c:\Users\JacopoBinati\anaconda3\envs\jacopo\lib\site-packages\xgboost\core.py", line 726, in inner_f
    return func(**kwargs)
  File "c:\Users\JacopoBinati\anaconda3\envs\jacopo\lib\site-packages\xgboost\sklearn.py", line 1491, in fit
    raise ValueError(
ValueError: Invalid classes inferred from unique values of `y`.  Expected: [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 3

## Random Forest Classifier

In [ ]:



param_grid = {
    'n_estimators': [50, 70],
    'max_depth': [20, 30],
    'min_samples_split': [2, 5],
    'min_samples_leaf': [1, 2]
}

rfr = RandomForestRegressor(
    random_state=42,
    criterion='mse',
    n_estimators=70,
    oob_score=True,
    n_jobs=-1
)

grid_search = GridSearchCV(
    estimator=rf,
    param_grid=param_grid,
    cv=5,
    scoring="neg_root_mean_squared_error",
    n_jobs=-1,
    verbose=4
)

grid_search.fit(X, y.values.ravel())

best_params = grid_search.best_params_
print("Best parameters found: ", best_params)

# Get the best estimator
best_rf = grid_search.best_estimator_

In [ ]:
y_holdout, X_holdout = dmatrices(formula, data_holdout, return_type='dataframe')

y_pred = best_rf.predict(X_holdout)


In [ ]:
mse = mean_squared_error(y_holdout, y_pred)
print("Mean Squared Error on holdout set: ", mse)

In [ ]:

cv_results = pd.DataFrame(grid_search.cv_results_)


# Metrics for the performance

In [ ]:
pivot_results = cv_results[
    ["param_max_features", "param_min_samples_split", "mean_test_score"]
].assign(
    mean_test_score=lambda x: -x["mean_test_score"],
    Variables=lambda x: x["param_max_features"],
    Min_nodes=lambda x: x["param_min_samples_split"]
).pivot(
    index="Min_nodes", columns="Variables", values="mean_test_score"
).round(2)

print("Cross-validation results:")
print(pivot_results)

## Feature Importance

In [ ]:
importances = best_rf.feature_importances_
feature_names = X.columns  # Get feature names from X
indices = np.argsort(importances)[::-1]

plt.figure(figsize=(12, 8))
plt.title("Feature Importances")
plt.bar(range(X.shape[1]), importances[indices])
plt.xticks(range(X.shape[1]), [feature_names[i] for i in indices], rotation=90)
plt.tight_layout()
plt.show()

# Print feature importances
print("\nFeature Importances:")
for f, imp in zip([feature_names[i] for i in indices], importances[indices]):
    print(f"{f}: {imp:.4f}")

####################################################################

importances_xgb = best_xgb.feature_importances_
feature_names_xgb = X.columns
indices_xgb = np.argsort(importances_xgb)[::-1]

plt.figure(figsize=(12, 8))
plt.title("Feature Importances XGB")
plt.bar(range(X.shape[1]), importances_xgb[indices_xgb])
plt.xticks(range(X.shape[1]), [feature_names_xgb[i] for i in indices_xgb], rotation=90)
plt.tight_layout()
plt.show()

# Print feature importances
print("\nFeature Importances XGB:")
for f, imp in zip([feature_names_xgb[i] for i in indices_xgb], importances_xgb[indices_xgb]):
    print(f"{f}: {imp:.4f}")
